# Natural Language Processing on US Streaming Hit Lyrics
## Data Collection

By Jeremias Figueroa

This passion project is an extension of the US Streaming Hits w/ Spotify Data project: https://github.com/jeremiasfv22/USA-Hits-Spotify-Data-Project. While the previous project analyzed the musical qualities of a song that entered the top 50 on streaming in the USA since March 2017, **this project analyzes the lyrical qualities of hit songs** that not only entered the top 50 on streaming but stayed there for enough weeks to **earn at least 72.5 million streams during their time on the top 50**.

If we take the top 100 songs per year (measured by how much streams it got during its time in the top 50 on HITS Magazine's Streaming chart) we see that the cutoff for the top 100 is around 72.5 million streams (using 2018-2021 data). 

Since 2017, I collected data from the HITS Daily Double Magazine on this Google Sheet: https://docs.google.com/spreadsheets/d/165OdLYjLt4AgeqP5S5PunRonDkpp28nueHLFv994bPk/edit?usp=sharing

This project utilizes various Python packages, such as gspread to connect Python with Google Sheets and Genius's API which hosts lyrics. 

The following Notebook collects songs from the Google Sheets above and looks for each song's respective lyrics. 

## Imports
First, install the following (add "-- upgrade" to upgrade the package if necessary). These packages are to be used in the following notebooks (for analytical and modeling purposes) as well. 

pip install pandas

pip install gspread

pip install lyricsgenius

pip install oauth2client

pip install df2gspread

pip install matplotlib

pip install seaborn

pip install sklearn

In [1]:
#to allow interaction with underlying operating system
import os

#to access Genius Lyrics
import lyricsgenius as lg

#to parse through spotipy's output
import json

#for data analysis
import numpy as np
import pandas as pd

#to remove the limits on a dataframe's columns displayed
pd.set_option('display.max_columns', None)

#to access Google Sheets with Python
import gspread
from google.oauth2 import service_account
from oauth2client.service_account import ServiceAccountCredentials

#to convert pd.DataFrames to Google Sheets
from df2gspread import df2gspread as d2g

#to ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Genius Lyrics API Credentials
Follow the instructions in the following page to create and register your application (needed to run this code and get lyrics): https://docs.genius.com/#/getting-started-h1. Make sure to save your access token.

It is recommendable practice to hide passwords from code. Therefore, environmental variables are to be created.

# CONVERT THE FOLLOWING TO CODE
#The following creates environmental variables
%env GENIUS_ACCESS_TOKEN=YOUR_GENIUS_ACCESS_TOKEN

In [2]:
%env GENIUS_ACCESS_TOKEN=CcwL2Jp9kNYNkCQ_9KH817_SymdlvxkNTdxZXiuCRoGG5dhsClKpgZc28oeocVtB

env: GENIUS_ACCESS_TOKEN=CcwL2Jp9kNYNkCQ_9KH817_SymdlvxkNTdxZXiuCRoGG5dhsClKpgZc28oeocVtB


In [3]:
genius_access_token = os.environ['GENIUS_ACCESS_TOKEN']

In [4]:
#create an instance of Genius Lyrics
gn = lg.Genius(genius_access_token)

# Importing the Google Sheet
Use song information from the following Google Sheet: https://docs.google.com/spreadsheets/d/165OdLYjLt4AgeqP5S5PunRonDkpp28nueHLFv994bPk/edit?usp=sharing

This Google Sheet is a collection of hit songs from HITS Daily Double, a music industry trade publication, and their weekly Overall Song Streams chart. The data includes how many times a song was streamed week-to-week for most weeks since 2017 (with the exception of holiday weeks, weeks when holiday music regularly push out non-holiday songs out of the top spots of music charts).

In order to be able to use this data on your own device, create a copy of the Google Sheet to your own Drive. Also, follow the instructions in the following link to connect Google Drive with Python: https://github.com/PrettyPrinted/youtube_video_code/tree/master/2021/10/14/How%20to%20Use%20Google%20Sheets%20With%20Python%20(2021)

For this python notebook, the file for our service account is the json file with the Google API keys downloaded from Google Developers Console. Please reference the link for more instructions.

In [5]:
#service account
sa = gspread.service_account(filename="gspread_service_account.json")

#sheet
sh = sa.open("HITS Streaming Songs")

#worksheet
wks = sh.worksheet("HITS Streaming Songs")

#uses all values in the worksheet for the data frame
gsheets_df = pd.DataFrame(wks.get_all_values())

header = gsheets_df.iloc[0] #isolate first row as header
gsheets_df = gsheets_df[1:] #get rid of header in original df
gsheets_df.columns = header

#set 'index' as index
gsheets_df = gsheets_df.set_index('index')

gsheets_df['title'] = gsheets_df['title'].str.rstrip(' ')
gsheets_df['artist'] = gsheets_df['artist'].str.rstrip(' ')

#drop christmas songs
gsheets_df.drop(gsheets_df[gsheets_df['holiday_music']=="1"].index,inplace=True)

#limit gsheets to the first five columns and remix_version (whether the desired song is a remix or not)
gsheets_df = pd.concat([gsheets_df.loc[:,'title':'streams_2017_to_present'],gsheets_df['remix_version']], axis=1)
    
#turn peak_date strings to date format
gsheets_df['peak_date'] = pd.to_datetime(gsheets_df['peak_date'])

#convert streams_2017_to_present to int
gsheets_df['streams_2017_to_present'] = gsheets_df['streams_2017_to_present'].str.replace(',','').astype('int')

#limit to only hit songs with more than 72.5 mil streams
gsheets_df = gsheets_df[gsheets_df['streams_2017_to_present'] > 72500000].reset_index(drop=True)

In [6]:
#to get the first item of a list
def get_first_list_item(df):
    for i in range(len(df)):
        df['title'][i] = df['title'][i][0]
    
#split at "feat." and grab first item to make it easier to search a song on Genius API
gsheets_df['title'] = gsheets_df['title'].str.split("feat.")
get_first_list_item(gsheets_df)

#split at "with" and grab first item to make it easier to search a song on Genius API
gsheets_df['title'] = gsheets_df['title'].str.split("with")
get_first_list_item(gsheets_df)

#to correct the change from above affecting 'break up with your girlfriend, i'm bored' by Ariana Grande
gsheets_df['title'][182] = "break up with your girlfriend, i'm bored"

#split at "Lil" and grab first item to make it easier to search a song on Genius API
gsheets_df['title'] = gsheets_df['title'].str.split("Lil")
get_first_list_item(gsheets_df)

#to correct the change from above affecting 'Lil Bit' by Nelly
gsheets_df['title'][391] = 'Lil Bit'

#split at "Drake" and grab first item to make it easier to search a song on Genius API
gsheets_df['title'] = gsheets_df['title'].str.split("Drake")
get_first_list_item(gsheets_df)

#split at "& Lil" and grab first item to make it easier to search a song on Genius API
gsheets_df['title'] = gsheets_df['title'].str.split("& Lil")
get_first_list_item(gsheets_df)

#split at "From" and grab first item to make it easier to search a song on Genius API
gsheets_df['title'] = gsheets_df['title'].str.split("From")
get_first_list_item(gsheets_df)

#split at "NEIKED" and grab first item to make it easier to search a song on Genius API
gsheets_df['title'] = gsheets_df['title'].str.split("NEIKED")
get_first_list_item(gsheets_df)

#split at "FEAT." and grab first item to make it easier to search a song on Genius API
gsheets_df['title'] = gsheets_df['title'].str.split("FEAT.")
get_first_list_item(gsheets_df)

#now split for dashes (with a space before it or else songs like '1-800-273-8255' will be affected)
gsheets_df['title'] = gsheets_df['title'].str.split(" -")
get_first_list_item(gsheets_df)

#getting rid of "(" and whitespace at the end of a song title
gsheets_df['title'] = gsheets_df['title'].str.strip("(")
gsheets_df['title'] = gsheets_df['title'].str.strip(" ")

#add " Remix" if a desired song is a remix
for i in range(len(gsheets_df)):
    if gsheets_df['remix_version'][i] == "1":
        gsheets_df['title'][i] = gsheets_df['title'][i] + " Remix"

#add dash back for 'Roses - Imanbek Remix' (there are numerous other popular remixes of this song)
gsheets_df['title'][282] = 'Roses - Imanbek Remix'
        
#add dash back for 'Savage Love (Laxed - Siren Beat)'
gsheets_df['title'][302] = 'Savage Love (Laxed - Siren Beat)'

#change lead artist on 'Pray for Me' to The Weeknd
gsheets_df['artist'][89] = 'Kendrick Lamar'

#change lead artist for "223's" to 9lokkNine
gsheets_df['artist'][230] = '9lokkNine'

#change lead artist of 'Savage Love (Laxed - Siren Beat)' to Jason Derulo
gsheets_df['artist'][302] = 'Jason Derulo'

#change lead artist of 'We Don't Talk About Bruno' to Carolina Gaitan
gsheets_df['artist'][432] = 'Carolina Gaitan'

gsheets_df

,title,artist,album,peak_date,streams_2017_to_present,remix_version
0,Shape Of You,Ed Sheeran,÷ (Deluxe),2017-03-03,175098946,0
1,Passionfruit,Drake,More Life,2017-03-17,126435791,0
2,Portland,Drake,More Life,2017-03-17,106312008,0
3,Tunnel Vision,Kodak Black,Painting Pictures,2017-03-17,98360268,0
4,Something Just Like This,The Chainsmokers,Memories...Do Not Open,2017-03-24,111814045,0
...,...,...,...,...,...,...
464,Cooped Up,Post Malone,Twelve Carat Toothache,2022-06-03,74296264,0
465,Late Night Talking,Harry Styles,Harry's House,2022-05-20,90151523,0
466,Running Up That Hill (A Deal With God),Kate Bush,Hounds Of Love (2018 Remaster),2022-06-03,119923607,0
467,Glimpse of Us,Joji,Glimpse of Us,2022-06-17,72996135,0


# Get lyrics from Genius Lyrics

In [7]:
df_for_func = gsheets_df.copy() #to ensure no changes are done to gsheets_df
list_of_lyrics = []

def lyrics_collector(df):
    """
    To append a sublist of song titles, artist names, and lyrics to list_of_jsons
    """
    song_title = df['title'][0] #grab just the first entry of the dataframe
    artist_name = df['artist'][0] #grab just the first entry of the dataframe
    song = gn.search_song(song_title + " " + artist_name)
    
    #if successful song will not be None
    if song != None:
        song_title = song.to_dict()["title_with_featured"]
        main_artist = song.to_dict()["artist"]
        lyrics = song.to_dict()["lyrics"]
        list_of_lyrics.append([song_title,main_artist,lyrics])
    else:
        list_of_lyrics.append([song_title,artist_name,"no_lyrics_found"])

In [8]:
#Genius API's connection goes out frequently; as a result, when acquiring lyrics run as the following
while len(df_for_func) > 0:
    while True:
        try:
            lyrics_collector(df_for_func)
            df_for_func = df_for_func.iloc[1:,:].reset_index(drop=True) #remove first item from df_for_func
            break
        except:
            pass

Searching for "Shape Of You Ed Sheeran"...
Done.
Searching for "Passionfruit Drake"...
Done.
Searching for "Portland Drake"...
Done.
Searching for "Tunnel Vision Kodak Black"...
Done.
Searching for "Something Just Like This The Chainsmokers"...
Done.
Searching for "HUMBLE. Kendrick Lamar"...
Searching for "HUMBLE. Kendrick Lamar"...
Searching for "HUMBLE. Kendrick Lamar"...
Done.
Searching for "LOVE. Kendrick Lamar"...
Done.
Searching for "DNA. Kendrick Lamar"...
Searching for "DNA. Kendrick Lamar"...
Done.
Searching for "LOYALTY. Kendrick Lamar"...
Done.
Searching for "ELEMENT. Kendrick Lamar"...
Done.
Searching for "I'm the One DJ Khaled"...
Done.
Searching for "Despacito Luis Fonsi"...
Done.
Searching for "XO Tour Llif3 Lil Uzi Vert"...
Searching for "XO Tour Llif3 Lil Uzi Vert"...
Searching for "XO Tour Llif3 Lil Uzi Vert"...
Done.
Searching for "Mask Off Future"...
Searching for "Mask Off Future"...
Done.
Searching for "1-800-273-8255 Logic"...
Done.
Searching for "That's What I L

Done.
Searching for "MIA Bad Bunny"...
Done.
Searching for "Shallow Lady Gaga"...
Done.
Searching for "Fine China Future"...
Done.
Searching for "Sunflower Post Malone"...
Searching for "Sunflower Post Malone"...
Done.
Searching for "when the party's over Billie Eilish"...
Done.
Searching for "Money Cardi B"...
Done.
Searching for "thank u, next Ariana Grande"...
Done.
Searching for "BAD! XXXTENTACION"...
Done.
Searching for "Armed And Dangerous Juice WRLD"...
Done.
Searching for "Going Bad Meek Mill"...
Done.
Searching for "Pure Cocaine Lil Baby"...
Done.
Searching for "Wow. Post Malone"...
Done.
Searching for "a lot 21 Savage"...
Done.
Searching for "Swervin A Boogie Wit da Hoodie"...
Done.
Searching for "Look Back at It A Boogie Wit da Hoodie"...
Done.
Searching for "Envy Me Calboy"...
Done.
Searching for "Dancing With A Stranger Sam Smith"...
Done.
Searching for "7 rings Ariana Grande"...
Searching for "7 rings Ariana Grande"...
Searching for "7 rings Ariana Grande"...
Done.
Search

Done.
Searching for "24 Money Man"...
Done.
Searching for "Dynamite BTS"...
Done.
Searching for "Heather Conan Gray"...
Done.
Searching for "my ex's best friend Machine Gun Kelly"...
Done.
Searching for "you broke me first Tate McRae"...
Done.
Searching for "Holy Justin Bieber"...
Specified song does not contain lyrics. Rejecting.
Searching for "Mr. Right Now Metro Boomin"...
Done.
Searching for "Dreams Fleetwood Mac"...
Done.
Searching for "positions Ariana Grande"...
Done.
Searching for "Forever After All Luke Combs"...
Done.
Searching for "Better Together Luke Combs"...
Done.
Searching for "34+35 Ariana Grande"...
Done.
Searching for "DÁKITI Bad Bunny"...
Done.
Searching for "Levitating Dua Lipa"...
Done.
Searching for "Therefore I Am Billie Eilish"...
Done.
Searching for "Body Megan Thee Stallion"...
Done.
Searching for "Somebody's Problem Morgan Wallen"...
Done.
Searching for "Cry Baby Megan Thee Stallion"...
Done.
Searching for "Whoopty CJ"...
Done.
Searching for "On Me Lil Baby"

In [9]:
#create a dataframe for lyrics and song+artist info (based on the Genius API info)
lyrics_df = pd.DataFrame(list_of_lyrics, columns=['song','artist','lyrics'])

## Correcting lyric data for songs that didn't successfully pass through the function above

For some reason, there are songs that don't have the desired song lyrics (some have wrong lyrics, results that aren't even lyrics, or no lyrics at all). Luckily, these problems are fixed when searching "artist + song" instead of "song + artist" 

In [10]:
#the following songs don't have the desired lyrics 
lyrics_to_clean_dct = {21: 'Issues', 22: 'Slide', 31:'Attention', 54:'New Rules',
    72:'Ric Flair Drip', 89:'Pray For Me', 229:'Circles', 238:'BOP',
    326: 'Holy', 339: 'Whoopty', 341: 'WITHOUT YOU', 367:'Ghost',
    424:'Easy On Me'}

#convert keys to list
lyrics_to_clean_indeces = list(lyrics_to_clean_dct.keys())


In [11]:
#the following is a subset of gsheets where the indexes are in lyrics_to_clean_indeces
df_to_switch_search_order = gsheets_df.iloc[lyrics_to_clean_indeces]
df_to_switch_search_order.reset_index(inplace=True) #to preserver index positions
df_to_switch_search_order

,index,title,artist,album,peak_date,streams_2017_to_present,remix_version
0,21,Issues,Julia Michaels,Nervous System,2017-05-05,84948119,0
1,22,Slide,Calvin Harris,Funk Wav Bounces Vol.1,2017-06-30,135059919,0
2,31,Attention,Charlie Puth,Voicenotes,2017-07-28,132167928,0
3,54,New Rules,Dua Lipa,Dua Lipa (Complete Edition),2017-12-29,149077936,0
4,72,Ric Flair Drip,Metro Boomin,Without Warning,2017-12-29,302337044,0
5,89,Pray For Me,Kendrick Lamar,Black Panther The Album Music From And Inspire...,2018-02-09,117571033,0
6,229,Circles,Post Malone,Hollywood's Bleeding,2019-09-06,667886259,0
7,238,BOP,DaBaby,KIRK,2019-09-27,319644736,0
8,326,Holy,Justin Bieber,Justice,2020-09-18,151985230,0
9,339,Whoopty,CJ,Loyalty Over Royalty,2021-01-15,177092351,0


In [12]:
#new list of jsons
list_of_lyrics_2 = []

def lyrics_collector(df):
    index = df['index'][0] #grab just the first index entry of the dataframe 
    song_title = df['title'][0] #grab just the first title entry of the dataframe
    artist_name = df['artist'][0] #grab just the first artist entry of the dataframe
    song = gn.search_song(artist_name + " " + song_title)
    if song != None:
        song_title = song.to_dict()["title_with_featured"]
        main_artist = song.to_dict()["artist"]
        lyrics = song.to_dict()["lyrics"]
        list_of_lyrics_2.append([index,song_title,main_artist,lyrics])
    else:
        list_of_lyrics_2.append([index,song_title,artist_name,"no_lyrics_found"])

while len(df_to_switch_search_order) > 0:
    while True:
        try:
            lyrics_collector(df_to_switch_search_order)
            df_to_switch_search_order = df_to_switch_search_order.iloc[1:,:].reset_index(drop=True)
            break
        except:
            pass

Searching for "Julia Michaels Issues"...
Done.
Searching for "Calvin Harris Slide"...
Done.
Searching for "Charlie Puth Attention"...
Done.
Searching for "Dua Lipa New Rules"...
Searching for "Dua Lipa New Rules"...
Searching for "Dua Lipa New Rules"...
Done.
Searching for "Metro Boomin Ric Flair Drip"...
Done.
Searching for "Kendrick Lamar Pray For Me"...
Done.
Searching for "Post Malone Circles"...
Done.
Searching for "DaBaby BOP"...
Done.
Searching for "Justin Bieber Holy"...
Done.
Searching for "CJ Whoopty"...
Done.
Searching for "The Kid LAROI WITHOUT YOU"...
Done.
Searching for "Justin Bieber Ghost"...
Done.
Searching for "Adele Easy On Me"...
Done.


In [13]:
lyrics_df_to_update = pd.DataFrame(list_of_jsons_2, columns=['index','song','artist','lyrics']).set_index('index')
lyrics_df_to_update

NameError: name 'list_of_jsons_2' is not defined

All lyrics are correct!

## Updating lyrics_df

In [ ]:
for i in range(len(lyrics_df)):
    if i in lyrics_to_clean_indeces:
        lyrics_df['lyrics'][i] = lyrics_df_to_update['lyrics'][i]

# Saving the dataframe to Google Sheets
The cells above take a long time to run! To save time for research purposes, the dataframe is to be converted to a Google Sheet (same link as the HITS Streaming Data Google Sheet) using the following code:

In [ ]:
#create a new df to save on Google Sheets (to make it easier to check if the song lyrics are correct)
lyrics_df.reset_index(inplace=True) #to save indeces
lyrics_df

In [ ]:
#save df to gsheets
url = "https://docs.google.com/spreadsheets/d/165OdLYjLt4AgeqP5S5PunRonDkpp28nueHLFv994bPk/edit#gid=0"
spreadsheet_key = url.split("/")[-2]
wks_name = "Top Songs' Lyrics"
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('gspread_service_account.json', scope)
gc = gspread.authorize(credentials)
d2g.upload(lyrics_df, spreadsheet_key, wks_name, credentials = credentials, row_names=False)

Success!!